#Using the Glove embedding df data sets...
##next steps


###-> extract text from pdf

###-> parse the cleaaned_text from resume with help of API enddpoint (Mistral AI)

###->storing skills, exp, yers of exp, previous job titles and doing similar embedding

###-> cosine similarity and recommendation

In [ ]:
print(df.columns)


Index(['qualifications', 'location', 'country', 'work type', 'preference',
       'job title', 'role', 'job description', 'benefits', 'skills',
       'responsibilities', 'company', 'min_salary', 'max_salary',
       'min_experience', 'max_experience', 'combined_text', 'glove_embedding'],
      dtype='object')


In [ ]:
#save the df as job_ebmedding.csv and aslosave it in my google drive
df.to_csv('job_ebmedding.csv', index=False)

In [ ]:
print(type(df['glove_embedding'][0]))
print(len(df['glove_embedding'][0]))  # Check the size of the embedding


<class 'numpy.ndarray'>
100


## Combine Embeddings into a Feature Matrix

In [ ]:
# Extract GloVe embeddings as features
text_embeddings = np.stack(df['glove_embedding'])

# Check the shape
print("Text Embeddings Shape:", text_embeddings.shape)


Text Embeddings Shape: (1615940, 100)


## Define Labels

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode job titles
label_encoder = LabelEncoder()
df['job_title_encoded'] = label_encoder.fit_transform(df['job title'])

# Verify encoding
print("Number of Classes:", len(label_encoder.classes_))


Number of Classes: 147


In [ ]:
#resume upload and extraction of text
!pip install pymupdf scikit-learn openai numpy pandas --quiet


In [ ]:
import fitz  # PyMuPDF

def extract_full_text_from_pdf(pdf_path):
    """Extracts full text from a PDF without any truncation."""
    doc = fitz.open(pdf_path)
    full_text = "\n".join(page.get_text("text") for page in doc)
    return full_text

# Example usage
pdf_path = "Resume.pdf"  # Replace with the actual file path
resume_text = extract_full_text_from_pdf(pdf_path)

print(resume_text)  # Print full resume text (optional)



SEHAJBIR SINGH 
416-890-6049 | sehaj12.12.2000@gmail.com | Toronto, ON | Linkedin.com/in/sehajbir-singh-542b34207 | Github.com/sehajbirsingh  
SKILLS 
 
Programming Languages: Python, SQL, Java, C# 
 
Data Engineering: Matillion, DBT, Airflow, Hadoop, Spark    
 
Data Warehousing and Databases: Snowflake, SQL 
Server, MySQL PostgreSQL, Database Staging 
 
DevOps & Version Control: Docker, Kubernetes, Jenkins, 
Git, GitHub
 
Data Science & Machine Learning: Pandas, NumPy, 
SciPy, Scikit-learn, TensorFlow, PyTorch
 
Data Visualization & Business Intelligence: Tableau, 
PowerBI, Plotly, Seaborn
 
Web Development: Flask, React, Node, HTML, CSS 
 
Agile Frameworks: Jira,Scrum, Kanban, Lean 
EXPERIENCE 
CLOUDEQS | REMOTE  
ASSOCIATE DATA ENGINEER | AUG, 2023 – JULY, 2024 
 
Led the end-to-end implementation of ETL/ELT processes using Matillion and DBT, incorporating data modeling strategies to 
optimize data flow and storage, achieving a 30% boost in processing efficiency and a 25% 

In [ ]:
!pip install langchain_huggingface --quiet

In [ ]:
!pip install openai --quiet

In [ ]:
import re

def clean_resume_text(resume_text):
    """Preprocess and optimize extracted resume text."""
    # Remove multiple spaces and special symbols
    resume_text = re.sub(r'\s+', ' ', resume_text)  # Replace multiple spaces/newlines with single space
    resume_text = re.sub(r'[^a-zA-Z0-9,.@|\s-]', '', resume_text)  # Remove non-ASCII characters
    resume_text = resume_text.strip()
    return resume_text

# Clean the extracted resume text
cleaned_resume = clean_resume_text(resume_text)

print(cleaned_resume)  # Check cleaned text before sending to Mistral


SEHAJBIR SINGH 416-890-6049 | sehaj12.12.2000@gmail.com | Toronto, ON | Linkedin.cominsehajbir-singh-542b34207 | Github.comsehajbirsingh SKILLS  Programming Languages Python, SQL, Java, C  Data Engineering Matillion, DBT, Airflow, Hadoop, Spark  Data Warehousing and Databases Snowflake, SQL Server, MySQL PostgreSQL, Database Staging  DevOps  Version Control Docker, Kubernetes, Jenkins, Git, GitHub  Data Science  Machine Learning Pandas, NumPy, SciPy, Scikit-learn, TensorFlow, PyTorch  Data Visualization  Business Intelligence Tableau, PowerBI, Plotly, Seaborn  Web Development Flask, React, Node, HTML, CSS  Agile Frameworks Jira,Scrum, Kanban, Lean EXPERIENCE CLOUDEQS | REMOTE ASSOCIATE DATA ENGINEER | AUG, 2023  JULY, 2024  Led the end-to-end implementation of ETLELT processes using Matillion and DBT, incorporating data modeling strategies to optimize data flow and storage, achieving a 30 boost in processing efficiency and a 25 reduction in latency.  Migrated of 100 of data from NetSui

In [ ]:
import json
from openai import OpenAI

# Initialize Hugging Face Mistral API using OpenAI-compatible structure
client = OpenAI(
    base_url="https://gj53iine8b0i8nmr.us-east-1.aws.endpoints.huggingface.cloud/v1/",
    api_key=""
)

def parse_resume(cleaned_text):
    """Extract structured resume data using Mistral API via OpenAI-compatible endpoint."""
    prompt = f"""
    Extract the following details from the provided resume text.
    Extract **only what is available**, and return empty values if missing.
    **Keep experience summary super concise.**

    **Return JSON format only.**

    Extract:
    - "skills": List of skills across resume seperated by','.
    - "experience_summary": Super concise work experience summary.
    - "years_of_experience": Total number of years worked related to experience section most probably you can calculate it will be near to job.
    - "previous_job_titles": List of past job titles.

    Resume Text:
    {cleaned_text}

    **Output only valid JSON. Example:**
    {{
        "skills": ["list", "of","all", "technical","non-technical","soft", "skills","accross", "resume","just list them","seperated by comma','"],
        "experience_summary": "brief overall summary of work experience (combined across different roles,NOT role-specific, NOT company-specific)...",
        "years_of_experience": integer,
        "previous_job_titles": ["list", "of", "previous", "roles"]
    }}
    """

    chat_completion = client.chat.completions.create(
        model="tgi",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500
    )

    # Extract response text
    response = chat_completion.choices[0].message.content

    if not response or "{" not in response:
        raise ValueError("Mistral API returned an empty or invalid response.")

    # Extract JSON from response
    json_start = response.find("{")
    json_end = response.rfind("}") + 1
    json_str = response[json_start:json_end]

    return json.loads(json_str)

# Send cleaned resume text for processing
structured_resume = parse_resume(cleaned_resume)

# Print extracted structured data
print(json.dumps(structured_resume, indent=4))


{
    "skills": [
        "Python",
        "SQL",
        "Java",
        "C",
        "Matillion",
        "DBT",
        "Airflow",
        "Hadoop",
        "Spark",
        "Snowflake",
        "SQL Server",
        "MySQL",
        "PostgreSQL",
        "Docker",
        "Kubernetes",
        "Jenkins",
        "Git",
        "GitHub",
        "Pandas",
        "NumPy",
        "SciPy",
        "Scikit-learn",
        "TensorFlow",
        "PyTorch",
        "Tableau",
        "PowerBI",
        "Plotly",
        "Seaborn",
        "Flask",
        "React",
        "Node",
        "HTML",
        "CSS",
        "Jira",
        "Scrum",
        "Kanban",
        "Lean"
    ],
    "experience_summary": "Data engineer and cloud analyst with experience in end-to-end data processing, migration, and ETL. Proficient in Matillion, DBT, Airflow, Snowflake, SQL, Python, Apache Spark, and data warehousing and databases.",
    "years_of_experience": 2,
    "previous_job_titles": [
        "A

In [ ]:
structured_resume

{'skills': ['Python',
  'SQL',
  'Java',
  'C',
  'Matillion',
  'DBT',
  'Airflow',
  'Hadoop',
  'Spark',
  'Snowflake',
  'SQL Server',
  'MySQL',
  'PostgreSQL',
  'Docker',
  'Kubernetes',
  'Jenkins',
  'Git',
  'GitHub',
  'Pandas',
  'NumPy',
  'SciPy',
  'Scikit-learn',
  'TensorFlow',
  'PyTorch',
  'Tableau',
  'PowerBI',
  'Plotly',
  'Seaborn',
  'Flask',
  'React',
  'Node',
  'HTML',
  'CSS',
  'Jira',
  'Scrum',
  'Kanban',
  'Lean'],
 'experience_summary': 'Data engineer and cloud analyst with experience in end-to-end data processing, migration, and ETL. Proficient in Matillion, DBT, Airflow, Snowflake, SQL, Python, Apache Spark, and data warehousing and databases.',
 'years_of_experience': 2,
 'previous_job_titles': ['Associate Data Engineer', 'Cloud Analyst']}

In [ ]:
import numpy as np

def create_resume_embedding(structured_resume, glove_embeddings, embedding_dim=100):
    """
    Combines skills and experience summary to create a meaningful text representation.
    Generates GloVe embeddings for the resume text.
    """
    # Combine skills and experience summary into a single text
    resume_text = " ".join(structured_resume["skills"]) + " " + structured_resume["experience_summary"]

    # Generate GloVe embedding for resume text
    resume_embedding = get_average_glove_vector(resume_text, glove_embeddings, embedding_dim)

    return resume_embedding

# Generate Resume Embedding
resume_embedding = create_resume_embedding(structured_resume, glove_embeddings, embedding_dim=100)


In [ ]:
resume_embedding

array([-1.99643418e-01,  1.32040665e-01,  1.59977600e-01, -1.69384688e-01,
        2.53064007e-01, -2.39086941e-01, -1.68464690e-01, -8.27152096e-03,
       -1.90627441e-01,  1.69569343e-01,  1.54127598e-01, -3.30863208e-01,
        1.38134003e-01,  1.36917129e-01,  2.29822129e-01, -1.48099527e-01,
        4.23784614e-01,  4.89554666e-02, -1.91212684e-01,  1.15380153e-01,
       -1.15878060e-01, -3.90677303e-02,  6.14817977e-01,  1.78011328e-01,
        9.44665633e-03, -1.46726474e-01,  9.10630077e-02, -6.24520108e-02,
       -2.11689323e-01, -8.95893015e-03, -2.34453827e-01,  3.68491322e-01,
       -4.15104359e-01,  4.70858663e-02,  1.13509996e-02,  9.17446837e-02,
        1.66983932e-01,  4.54507470e-01,  1.26742661e-01, -2.65792459e-01,
       -1.47465974e-01, -5.28786719e-01, -2.23318279e-01, -1.11353531e-01,
       -1.31944865e-01, -2.07045320e-02,  2.56679982e-01, -3.79188329e-01,
        4.60472070e-02, -4.79145110e-01,  1.17941216e-01, -1.45100360e-03,
        5.38558625e-02,  

In [ ]:
#copy df
df_copy = df.copy()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Convert job embeddings DataFrame to NumPy array
job_embeddings = np.stack(df_copy['glove_embedding'].values)  # Shape: (1615940, 100)

# Compute cosine similarity between resume & all job embeddings
similarities = cosine_similarity([resume_embedding], job_embeddings)[0]

# Add similarity scores to the DataFrame
df_copy['similarity_score'] = similarities

# UI Components
# Multi-select dropdown for country filtering
country_options = sorted(df_copy['country'].unique())
country_selector = widgets.SelectMultiple(
    options=country_options,
    value=[country_options[0]],
    description='Countries',
    layout={'width': '50%'}
)

# Slider for number of jobs to display
num_jobs_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=50,
    step=1,
    description='Jobs to show:'
)

# Button to fetch recommendations
recommend_button = widgets.Button(
    description='Get Job Recommendations',
    button_style='success'
)

# Output widget for displaying results
output = widgets.Output()

def update_recommendations(b):
    with output:
        output.clear_output()
        selected_countries = list(country_selector.value)
        num_jobs = num_jobs_slider.value

        # Filter jobs based on selected countries
        filtered_jobs = df_sorted[df_sorted['country'].isin(selected_countries)]

        # Display top N matching jobs
        display(filtered_jobs[['job title','company','qualifications', 'location', 'country','work type',]].head(num_jobs))


recommend_button.on_click(update_recommendations)

# Display UI
display(country_selector, num_jobs_slider, recommend_button, output)

SelectMultiple(description='Countries', index=(0,), layout=Layout(width='50%'), options=('Afghanistan', 'Alban…

IntSlider(value=5, description='Jobs to show:', max=50, min=1)

Button(button_style='success', description='Get Job Recommendations', style=ButtonStyle())

Output()